In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# constants
L   = 1.0   # 1-D computational domain size
ghost_zone = 1

In [ ]:
def xy(N):
    x = np.linspace( 0.0, L, N+2*ghost_zone ) 
    y = np.linspace( 0.0, L, N+2*ghost_zone )
    return x,y

In [ ]:
# define a reference analytical solution
def u_ref(N):
    x = np.linspace( 0.0, L, N+2*ghost_zone ) 
    y = np.linspace( 0.0, L, N+2*ghost_zone )

    ref_func = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
    for i in range(N+2*ghost_zone):
        for j in range(N+2*ghost_zone):
            ref_func[i,j] = np.sin(x[i])*np.sin(y[j])+1
    return ref_func

#boundary condition
def source_func(N):
    x = np.linspace( 0.0, L, N+2*ghost_zone ) 
    y = np.linspace( 0.0, L, N+2*ghost_zone )

    rho = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
    for i in range(N+2*ghost_zone):
        for j in range(N+2*ghost_zone):
            rho[i,j] = -2*np.sin(x[i])*np.sin(y[j]) 
    return rho

def initial_bound(N):
    
    ib = source_func(N)
    ur = u_ref(N)
    ib[:ghost_zone,:] =  ur[:ghost_zone,:]
    ib[-ghost_zone:,:] =  ur[-ghost_zone:,:]
    ib[:,:ghost_zone] =  ur[:,:ghost_zone]
    ib[:,-ghost_zone:] =  ur[:,-ghost_zone:]

    return ib

In [ ]:
N=3

In [ ]:
b = source_func(N)
x = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
A = np.diagflat( np.ones(N+2*ghost_zone-1)*(-1),       -1 ) + \
    np.diagflat( np.ones(N+2*ghost_zone)*(4), 0 ) + \
    np.diagflat( np.ones(N+2*ghost_zone-1)*(-1),       +1 );

r = b - np.matmul(A,x)
d = r

In [ ]:
for k in range(10):
      
    alpha = np.matmul(r,r.T)/(d*A*d)
    
    x = x + alpha*d
    
    rnew = r - alpha*A*d

    beta = np.matmul(rnew.T,rnew)/np.matmul(r.T,r)
    
    d = r + beta*d
    r = rnew